In [ ]:
from smolagents import CodeAgent, LiteLLMModel
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

In [ ]:
model = LiteLLMModel(model_id="openai/gpt-4.1-mini") # Could use 'gpt-4o'

### Code Agent

In [ ]:
code_agent = CodeAgent(tools=[], model=model, add_base_tools=True)

In [ ]:
code_agent.run(
    "Could you give me the 118th number in the Fibonacci sequence?",
)

In [ ]:
code_agent.run(
    "Could you get me the title of the page at url 'https://huggingface.co/blog'?",
)

In [ ]:
code_agent.memory.get_full_steps()
code_agent.memory.steps[1].model_input_messages

#code_agent.logs[2]
#pprint.pprint(code_agent.logs[0].system_prompt)

### Tool Agent

In [ ]:
from smolagents import ToolCallingAgent

tool_agent = ToolCallingAgent(tools=[], model=model, add_base_tools=True)
tool_agent.run("Could you get me the title of the page at url 'https://huggingface.co/blog'?")

### MCP Agent

In [ ]:
from smolagents import ToolCollection, CodeAgent
from mcp import StdioServerParameters

In [ ]:
server_parameters = StdioServerParameters(
    command="npx",
    args=["@gongrzhe/server-gmail-autoauth-mcp"],
    env={"UV_PYTHON": "3.13", **os.environ},
)


with ToolCollection.from_mcp(server_parameters, trust_remote_code=True) as tool_collection:
    agent = CodeAgent(tools=[*tool_collection.tools], add_base_tools=True, model=model)
    agent.run("""Help me to find email send by TTBank in last 3 days, ("""
              """if you can find it, reply thanks to TTBank!""")

In [ ]:
server_parameters = StdioServerParameters(
        command="docker",
        args=[
            "run",
            "-i",
            "--rm",
            "-e",
            "GITHUB_PERSONAL_ACCESS_TOKEN",
            "ghcr.io/github/github-mcp-server"
        ],
        env={
            "GITHUB_PERSONAL_ACCESS_TOKEN": os.getenv("GITHUB_PERSONAL_ACCESS_TOKEN"),
            **os.environ,
        },
    )

prompt = """Could you locate my (jeff52415) repository named leetcode-cookbook, please use github tool instead of searching on github.com
I'm currently practicing LeetCode by adding one new example per day to my `leetcode-cookbook` repository.

Do the following without asking for confirmation:

- Review the existing repository structure
- Select and add **one new LeetCode problem solution** that is not already present in the repo
- Place the solution in the appropriate category folder under `problems/`
- Use the filename format: `<id>-<slug>-<difficulty>.py`
- At the top of the file, include a comment block with:
  - Title
  - ID
  - Difficulty
  - Link
  - A short human-style note (e.g., what this problem reinforced or why it's interesting)
- Implement the solution using a `Solution` class and appropriate methods
- Include inline comments explaining the approach, trade-offs, and time/space complexity

Example format:

```python
ID: 198
Title: House Robber
Difficulty: Medium
Link: https://leetcode.com/problems/house-robber/

Note: Practicing daily — today’s problem is a classic DP case.


class Solution:
    def rob(self, nums: List[int]) -> int:
        # dp[i] = max money that can be robbed up to index i
        # Transition: dp[i] = max(dp[i-1], dp[i-2] + nums[i])
        # Time: O(n), Space: O(1)
        pass  # TODO: implement

        
    
Then create a branch then make a pull request.


"""
with ToolCollection.from_mcp(server_parameters, trust_remote_code=True) as tool_collection:
    agent = CodeAgent(tools=[*tool_collection.tools], add_base_tools=True, model=model)
    agent.run(prompt)

In [ ]:
with ToolCollection.from_mcp(server_parameters, trust_remote_code=True) as tool_collection:
    agent = CodeAgent(tools=[*tool_collection.tools], add_base_tools=True, model=model)
    agent.run("""Show me the github repo structure of my project: leetcode-cookbook, use github tool, the url is: https://github.com/jeff52415/leetcode-cookbook/tree/main""")

In [ ]:
with ToolCollection.from_mcp({"url": "http://127.0.0.1:8000/sse"}, trust_remote_code=True) as tool_collection:
    agent = CodeAgent(tools=[*tool_collection.tools], add_base_tools=True)
    agent.run("Please find a remedy for hangover.")

### Tracinig

In [ ]:
#!python -m phoenix.server.main serve
from phoenix.otel import register
from openinference.instrumentation.smolagents import SmolagentsInstrumentor

register()
SmolagentsInstrumentor().instrument()

In [ ]:
from smolagents import (
    CodeAgent,
    ToolCallingAgent,
    WebSearchTool,
    VisitWebpageTool,
    InferenceClientModel,
)

model = InferenceClientModel()

search_agent = ToolCallingAgent(
    tools=[WebSearchTool(), VisitWebpageTool()],
    model=model,
    name="search_agent",
    description="This is an agent that can do web search.",
)

manager_agent = CodeAgent(
    tools=[],
    model=model,
    managed_agents=[search_agent],
)
manager_agent.run(
    "If the US keeps its 2024 growth rate, how many years will it take for the GDP to double?"
)

### Tool

In [ ]:
#from langchain.agents import load_tools
from langchain_community.tools import ShellTool

shell_ = ShellTool()
#shell_tool = Tool.from_langchain(shell_,)